# stasmodelsの事例

重回帰を行うときに、多重共線性の影響を見る例です。

参考：[Pythonでライトに始める統計分析](https://tech.prog-8.com/entry/2020/12/21/090434)

これは、ある仮想野球チームのトップ２０年俸プレーヤの成績データを解析する。

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/miwamasa/DataScience2024/main/notebooks/data/teamG.csv')
df

In [ ]:
df.columns

各コラムは以下の意味を持つとする。
- Name:名前
- AnnualSalary:年防
- HomeRuns:本塁打数
- Walks:四死球
- Steels:盗塁数
- RangeFactor:守備率
- GamePlayed:出場試合数
- YearsOfCareer:年数
- ForeignPlayer:外国人選手かどうか
- FrancisePlayer:生え抜き線種かどうか

In [ ]:
df.describe()

Parra選手はデータが登録されていないので除きます。

In [ ]:
df=df.dropna()

データ型をしらべてみます。

In [ ]:
df.dtypes

HomeRunsが数値と認識されていなかったので強制的に数値にします。

In [ ]:
df.HomeRuns = df.HomeRuns.apply(pd.to_numeric)

In [ ]:
df.dtypes

### 回帰分析1
線形モデルを仮定していたので statsmodels.regression.linear_model.OLS を使います。年俸を被説明変数、その他を説明変数とします。また、定数項も追加します

In [ ]:
Y = df[df.columns[1]] # 目的変数はAnnualSalary
X = df[df.columns[2:]] # 説明変数はその他

X = sm.add_constant(X) # 切片項も追加
model = sm.OLS(Y, X)
  
results = model.fit()
print(results.summary())

重回帰分析をするときに最もよく使う指標の一つであるVIF(Variance Inflation Factor)を算出してみようと思います。重回帰分析をする時に説明変数同士が相関していると**多重共線性**という問題を引き起こしますが、VIFを算出することで多重共線性を起こしている可能性のある変数を発見することができます。

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

col_count = model.exog.shape[1]

vifs = [variance_inflation_factor(model.exog, i) for i in range(1, col_count)]
pd.DataFrame(vifs, index=model.exog_names[1:], columns=["VIF"])

基準は、5,10とされていることから、Walksが外した方がよいことになる

HomeRunsとWalksの相関をみます。強打者は四死球うけやすい？

In [ ]:
df[['HomeRuns', 'Walks']].corr()

相関がありましたので、Walksを除いて回帰をします。

In [ ]:
df1 = df.drop(['Walks'], axis=1)

Y = df1[df1.columns[1]]
X = df1[df1.columns[2:]]

X = sm.add_constant(X)
model = sm.OLS(Y, X)
  
results = model.fit()
print(results.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

col_count = model.exog.shape[1]

vifs = [variance_inflation_factor(model.exog, i) for i in range(1, col_count)]
pd.DataFrame(vifs, index=model.exog_names[1:], columns=["VIF"])

In [ ]:
fig = sm.graphics.plot_partregress_grid(results)
#fig.tight_layout(pad=1.0)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ax.title.set_color('white')

plt.style.use('default')
sm.graphics.plot_partregress('AnnualSalary', 'HomeRuns', ['Steels', 'RangeFactor',
       'GamesPlayed', 'YearsOfCareer', 'ForeignPlayer', 'FrancisePlayer'], ax=ax,
                                data=df1, obs_labels=False);

In [ ]:
df1.columns

In [ ]:
import seaborn as sns
sns_plot = sns.pairplot(df1, hue="AnnualSalary", diag_kind='hist')